# Main notebook - experimenting with guesses

SPDX-License-Identifier: 0BSD

This does semantic search based solely on repository name and user input.

Compare to
[`findrepo`](https://github.com/EliahKagan/newrepo-findrepo#using-findrepo).

In [1]:
import logging

import numpy as np
from tabulate import tabulate

import fr2ex

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
names = fr2ex.remote.fetch_repo_names()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.6p1)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [4]:
len(names)

685

In [5]:
moderation = fr2ex.moderation.get_moderation(names)

INFO:root:Reading cached moderation.


In [6]:
len(moderation)

685

In [7]:
any(result['flagged'] for result in moderation)

False

In [8]:
fr2ex.tokens.report_cost(names)

It looks like the rate is $0.0004 per 1000 tokens. If so, the cost to
process 1944 tokens is about $0.0007776 (that is, 0.0777600 cents).


In [9]:
embeddings = fr2ex.embedding.embed_many(names)

INFO:root:Reading cached embeddings.


In [10]:
def guess(name: str, count: int = 5) -> None:
    """Show top guesses for similarity of name to already embedded names."""
    scores = embeddings @ fr2ex.embedding.embed(name)
    ordering = sorted(zip(scores, names), reverse=True)
    table = [(name, score) for score, name in ordering]
    return tabulate(table[:count], tablefmt='html', floatfmt='.6f')

In [11]:
guess('algorithm')

Calculator,0.868787
Geometry,0.865275
algorithms-suggestions,0.860121
Alias,0.851815
crystal-algo,0.849683


In [12]:
guess('sorting')

SortingRanges,0.932616
Sorts,0.909054
sortkey,0.859548
PartialSort,0.855047
tsort,0.840450


In [13]:
guess('algorithm-visualization')

algorithms-suggestions,0.888990
algorithms-python,0.859365
crystal-algo,0.855845
Graph-gist,0.842326
codegraph,0.833072


In [14]:
guess('maven')

try-maven,0.915951
mavener,0.879947
retry-maven,0.868217
pimpl,0.836310
try-gradle,0.833699


In [15]:
guess('find-repo')

newrepo-findrepo,0.916256
findrepo2-experiment,0.867356
backup-repos,0.865059
find-articles,0.856952
TestRepo,0.842280


In [16]:
guess('graph theory', count=10)

codegraph,0.881156
Graph,0.879309
Graph-gist,0.864633
Geometry,0.839598
wpf-graph,0.829072
graphyaml,0.820667
gh-profile,0.814757
c-sketches,0.813895
trig,0.809151
Draw,0.808803


In [17]:
guess('shortest paths', count=10)

Dijkstra,0.891422
Paths,0.838905
TreeTraversal,0.830679
Kruskal,0.816463
algorithms-suggestions,0.816262
Traverse,0.813065
Permutations,0.806660
treejs,0.801425
DFS,0.800874
kt-short,0.800389


In [18]:
guess('shortest-paths', count=10)

Dijkstra,0.892523
Paths,0.836284
TreeTraversal,0.836171
Traverse,0.819795
algorithms-suggestions,0.819142
Kruskal,0.814032
type-arrows,0.812889
kt-short,0.811228
sequence-dag,0.809261
dot-and-cross,0.806277


In [19]:
guess('graph traversal', count=10)

TreeTraversal,0.897430
Traverse,0.869958
Graph,0.868196
codegraph,0.856913
Graph-gist,0.850885
TraverseTree,0.850041
TreeTraversalAnimations,0.844913
Dijkstra,0.825463
DFS,0.823535
sequence-dag,0.819539


In [20]:
guess('graph-traversal', count=10)

TreeTraversal,0.904239
Traverse,0.879972
Graph,0.878319
Graph-gist,0.875122
codegraph,0.872254
TraverseTree,0.865488
TreeTraversalAnimations,0.851134
Dijkstra,0.842995
wpf-graph,0.840984
sequence-dag,0.837127


In [21]:
guess('graphics', count=10)

Graph,0.889270
printing,0.882984
Geometry,0.881480
Draw,0.875411
fps,0.859528
works,0.848890
coordinates,0.845146
Components,0.842697
interop,0.841286
Benchmark,0.840062


In [22]:
guess('deluge')

Flood,0.864972
foobar,0.825892
lambda-demo,0.819254
Dynamo,0.816752
yule,0.814018


In [23]:
guess('source control')

git,0.822964
Linkage,0.807198
gitscripts,0.806762
capture.net,0.803081
test-site,0.801783


In [24]:
guess('source-control')

git,0.831532
git-notes,0.830648
dot-imports,0.815999
gitscripts,0.813048
editor-configs,0.812764
